# Instructions
- Put the sqlite file into the /data folder of this repo

In [ ]:
from cellpaint_database.database_loader import DatabaseLoader
import matplotlib.pyplot as plt

db_path = "/Users/thomasathey/Documents/shavit-lab/fraenkel/data/aneesh/als_cellpainting_relative.sqlite" # path to database
data_root = "/Users/thomasathey/Documents/shavit-lab/fraenkel/data/aneesh" # path to the data directory

In [ ]:
dbl = DatabaseLoader(db_path, data_root)

In [ ]:
plate = 2
well = ('B', 2, 1) # ('row', column, field)
well2 = 1 # alternative way to specify well - site number
channel = 1

line = dbl.get_line(plate, well, channel)
treatment = dbl.get_treatment(plate, well, channel)
img = dbl.get_image(plate, well2, channel)

In [ ]:
plt.imshow(img)
plt.title(f"{line} x {treatment}")